# Génération du fichier périmètre

Ce script a pour but de générer le fichier périmètre du partenaire après injection des Bordereaux dans RenoiRH RECR.
Dans le répertoire HR se trouve les différentes extractions relatives à chaque lot.
Dans le répertoire BDC, se trouve les différentes extractions des informations envoyés par le partenaire dans la Base de collecte.

## Extraction des données

Pour le MAA, le script d'extraction HR se trouve dans le répertoire */data/scripts/CheckList_MAA* . Il faut lancer *sql @extract_AG0xCET.sql*


Il faut déposer les fichiers extraits dans le répertoire HR.

Pour les fichiers à déposer dans le répertoire BDC.
Pour controler le lot AG01 : Il faut déposer le fichier *onp_file_matricules.csv*. 



Pour contrôler le lot AG02 : il faut lancer la commande */opt/lampp/bin/mysql -u MCC -pMCC MAA_500 <PERIMETRE_CPT_DOSSIER_ZY0F.sql>/exploit/hra/txt/tmp/PERIMETRE_CPT_DOSSIER_ZY0F.csv*



Pour contrôler le lot AG03 : il faut lancer la commande */opt/lampp/bin/mysql -u MCC -pMCC MAA_500 <PERIMETRE_CPT_DOSSIER_ZYCOv2.sql>/exploit/hra/txt/tmp/PERIMETRE_CPT_DOSSIER_ZYCO.csv*


Pour contrôler le lot AG04 : il faut lancer la commande */opt/lampp/bin/mysql -u MCC -pMCC MAA_500 <PERIMETRE_CPT_DOSSIER_ZYGR.sql>/exploit/hra/txt/tmp/PERIMETRE_CPT_DOSSIER_ZYGR.csv*

Pour contrôler le lot AG05 : il faut lancer la commande */opt/lampp/bin/mysql -u MCC -pMCC MAA_500 <PERIMETRE_CPT_DOSSIER_ZYPO.sql>/exploit/hra/txt/tmp/PERIMETRE_CPT_DOSSIER_ZYPO.csv*

Il faut télécharger les fichiers csv généré dans le répertoire BDC


## algorithme de génération du fichier périmètre

Pour chaque ligne :
Alimenter la colonne Matricule BDC avec P_CIV_MATCLE et la colonne MAtricule HR avec matcle_init.
Pour la colonne **LOT 1 - Création de l'agent dans RENOIRH** rechercher le matricule HR dans /HR/AG01AGR.csv : si le matricule est trouvé mettre __OK__ sinon __KO__

Pour la colonne **LOT 2 - Adresses** rechercher le matricule HR dans /BDC/PERIMETRE_CPT_DOSSIER_ZY0F.csv *ET* dans /HR/AG02AGR.csv : si le matricule est trouvé dans les 2 fichiers mettre __OK__ sinon __KO__

Pour la colonne **LOT 2 - Contrats** rechercher le matricule HR dans /BDC/PERIMETRE_CPT_DOSSIER_ZYCO.csv, si le matricule HR est non trouvé mettre __NC__ et passer à la suite. Si le matricule est trouvé alors rechercher le matricule HR dans /HR/AG02COAGR.csv  si le matricule est trouvé dans les 2 fichiers mettre __OK__ sinon __KO__

Pour la colonne **LOT 3 - Carrières** rechercher le matricule HR dans /BDC/PERIMETRE_CPT_DOSSIER_ZYGR.csv si le matricule HR est non trouvé mettre __NC__ et passer à la suite. Si le matricule est trouvé alors rechercher le matricule HR dans /HR/AG03AGR.csv  si le matricule est trouvé dans les 2 fichiers mettre __OK__ sinon __KO__

Pour la colonne **LOT 4 - Positions** rechercher le matricule HR dans /BDC/PERIMETRE_CPT_DOSSIER_ZYPO.csv *ET* dans /HR/AG04AGR.csv : si le matricule est trouvé dans les 2 fichiers mettre __OK__ sinon __KO__

Pour la colonne **LOT 5 - Absences** rechercher le matricule HR dans /BDC/PERIMETRE_CPT_DOSSIER_ZYAG.csv *ET* dans /HR/AG05AGR.csv : si le matricule est trouvé dans les 2 fichiers mettre __OK__ sinon __KO__

In [ ]:
 import os
os.environ ['PATH'] 
__version__ = '1.0.1'

In [ ]:
import xlwings as xw
import numpy as np
import pandas as pd
import datetime as dt
import time
import sys
from datetime import time, tzinfo, timedelta
class GMT1(tzinfo):
    def utcoffset(self, dt):
        return timedelta(hours=1)
    def dst(self, dt):
        return timedelta(0)
    def tzname(self,dt):
        return "Europe/Paris"
t= dt.datetime.now()
print("Début du traitement à ",t.strftime("%H:%M:%S %Z"))


In [ ]:
t= dt.datetime.now()
print("Création du fichier périmetre à ",t.strftime("%H:%M:%S %Z"))
print ("Création du classeur ")
wbPerimetre = xw.Book()  # Ceci va créer un nouveau classeur PERIMETRE
# création de la feuille Périmetre
feuillePerimetre = wbPerimetre.sheets[0]

# Alimentation de l'entête du fichier PERIMETRE
feuillePerimetre.range('A1').value = ['Matricule BDC','CORPS','GRADE','Statut','','','Périmètre reprise','Matricule RENOIRH','LOT 1 - Création agent dans RENOIRH','LOT 2 - Adresses','LOT 2 - Contrats','LOT 3 - Carrières','LOT 4 - Positions','PROCHE - Conjoint','PROCHE - Enfant','LOT 5 - Absences']

In [ ]:
# lecture du fichier /BDC/onp_file_matricules.csv
total_matricules = pd.read_csv('BDC/onp_file_matricules.csv',sep=';',header=0,usecols=['P_CIV_MATCLE','matcle'],encoding='utf-8')

# Alimentation de 'Matricule BDC' et de 'Matricule RENOIRH'
feuillePerimetre.range('A1').options(index=False).value =pd.DataFrame({'Matricule BDC': total_matricules['P_CIV_MATCLE']})
feuillePerimetre.range('H1').options(index=False).value =pd.DataFrame({'Matricule RENOIRH': total_matricules['matcle']})

Pour chaque ligne : Alimenter la colonne Matricule BDC avec P_CIV_MATCLE et la colonne MAtricule HR avec matcle_init. 
**Pour la colonne LOT 1** - Création de l'agent dans RENOIRH : rechercher le matricule HR dans /HR/AG01AGR.csv : si le matricule est trouvé mettre OK sinon KO


In [ ]:
# lecture du fichier /BDC/onp_file_matricules.csv
total_matricules_bdc = pd.read_csv('BDC/onp_file_matricules.csv',sep=';',header=0,usecols=['matcle'])

# Lecture des matricules chargés dans RenoiRH
AG01_RenoiRH = pd.read_csv('HR/MAA_AG01.csv',header=None,names=('matcle','OK'))
AG01_RenoiRH['OK']='OK'
# print(AG01_RenoiRH)
# print(total_matricules_bdc)
crit_AG01_OK = total_matricules_bdc.matcle.isin(AG01_RenoiRH.matcle)
total_matricules_bdc['LAG1']=crit_AG01_OK*1
total_matricules_bdc['LOT 1 - Création agent dans RENOIRH']=total_matricules_bdc.replace([0, 1, 2], ['KO', 'OK', 'NC'])['LAG1']
feuillePerimetre.range('I1').options(index=False).value =total_matricules_bdc['LOT 1 - Création agent dans RENOIRH']


Pour la colonne **LOT 2 - Adresses** rechercher le matricule HR dans /BDC/PERIMETRE_CPT_DOSSIER_ZY0F.csv *ET* dans /HR/AG02AGR.csv : si le matricule est trouvé dans les 2 fichiers mettre __OK__ sinon __KO__

In [ ]:
# lecture du fichier /BDC/PERIMETRE_CPT_DOSSIER_ZY0F.csv
total_matricules_adr = pd.read_csv('BDC/PERIMETRE_CPT_DOSSIER_ZY0F.csv',sep=';',header=0,usecols=['matcle'])
# total_matricules_adr['KO']='KO'
# Lecture des matricules chargés dans RenoiRH
AG02_RenoiRH = pd.read_csv('HR/MAA_AG02_0F.csv',header=None,names=('matcle','OK'))
# AG02_RenoiRH['OK']='OK'

crit_AG02_OK = crit_AG01_OK & total_matricules_adr.matcle.isin(AG02_RenoiRH.matcle)
crit_AG020F_NC = total_matricules_bdc.matcle.isin(total_matricules_adr.matcle)
crit_AG020F_OK = total_matricules_bdc.matcle.isin(AG02_RenoiRH.matcle) & crit_AG01_OK
total_matricules_bdc['0FNC']= crit_AG020F_NC
total_matricules_bdc['0FOK']= crit_AG020F_OK
total_matricules_bdc['L20F']= (total_matricules_bdc['0FNC']*1 + total_matricules_bdc['0FOK']*1)

# total_matricules_adr['L20F']=crit_AG02_OK * 1
total_matricules_bdc['LOT 2 - Adresses']=total_matricules_bdc.replace([0, 1, 2], ['NC', 'KO', 'OK'])['L20F']
feuillePerimetre.range('J1').options(index=False).value =total_matricules_bdc['LOT 2 - Adresses']


Pour la colonne **LOT 2 - Contrats** rechercher le matricule HR dans /BDC/PERIMETRE_CPT_DOSSIER_ZYCO.csv, si le matricule HR est non trouvé mettre __NC__ et passer à la suite. Si le matricule est trouvé alors rechercher le matricule HR dans /HR/AG02COAGR.csv  si le matricule est trouvé dans les 2 fichiers mettre __OK__ sinon __KO__

In [ ]:
# lecture du fichier /BDC/PERIMETRE_CPT_DOSSIER_ZYCO.csv
total_matricules_con = pd.read_csv('BDC/PERIMETRE_CPT_DOSSIER_ZYCO.csv',sep=';',header=0,usecols=['matcle'])
# total_matricules_adr['KO']='KO'
# Lecture des matricules chargés dans RenoiRH
AG02_RenoiRH = pd.read_csv('HR/MAA_AG02_CO.csv',header=None,names=('matcle','OK'))
# AG02_RenoiRH['OK']='OK'

# création d'une série vérifiant la présence de BdC dans HR
crit_AG02CO_NC = total_matricules_bdc.matcle.isin(total_matricules_con.matcle)
crit_AG02CO_OK = total_matricules_bdc.matcle.isin(AG02_RenoiRH.matcle) & crit_AG01_OK
total_matricules_bdc['CONC']= crit_AG02CO_NC
total_matricules_bdc['COOK']= crit_AG02CO_OK
total_matricules_bdc['L2CO']= (total_matricules_bdc['CONC']*1 + total_matricules_bdc['COOK']*1)
# 
total_matricules_bdc['LOT 2 - Contrats']=total_matricules_bdc.replace([0, 1, 2], ['NC', 'KO', 'OK'])['L2CO']
# print(total_matricules_bdc)
feuillePerimetre.range('K1').options(index=False).value =total_matricules_bdc['LOT 2 - Contrats']


Pour la colonne **LOT 3 - Carrières** rechercher le matricule HR dans /BDC/PERIMETRE_CPT_DOSSIER_ZYGR.csv si le matricule HR est non trouvé mettre __NC__ et passer à la suite. Si le matricule est trouvé alors rechercher le matricule HR dans /HR/AG03AGR.csv  si le matricule est trouvé dans les 2 fichiers mettre __OK__ sinon __KO__


In [ ]:
# lecture du fichier /BDC/PERIMETRE_CPT_DOSSIER_ZYGR.csv
total_matricules_car = pd.read_csv('BDC/PERIMETRE_CPT_DOSSIER_ZYGR.csv',sep=';',header=0,usecols=['matcle'])

# Lecture des matricules chargés dans RenoiRH
AG02_RenoiRH = pd.read_csv('HR/MAA_AG03.csv',header=None,names=('matcle','OK'))
# AG02_RenoiRH['OK']='OK'

# création d'une série vérifiant la présence de BdC dans HR
crit_AG03AGR_NC = total_matricules_bdc.matcle.isin(total_matricules_car.matcle)
crit_AG03AGR_OK = total_matricules_bdc.matcle.isin(AG02_RenoiRH.matcle) & crit_AG01_OK
total_matricules_bdc['AGNC']= crit_AG03AGR_NC
total_matricules_bdc['AGOK']= crit_AG03AGR_OK
total_matricules_bdc['L3GR']= (total_matricules_bdc['AGNC']*1 + total_matricules_bdc['AGOK']*1)
# 
total_matricules_bdc['LOT 3 - Carrières']=total_matricules_bdc.replace([0, 1, 2], ['NC', 'KO', 'OK'])['L3GR']
# print(total_matricules_bdc)
feuillePerimetre.range('L1').options(index=False).value =total_matricules_bdc['LOT 3 - Carrières']


Pour la colonne **LOT 4 - Positions** rechercher le matricule HR dans /BDC/PERIMETRE_CPT_DOSSIER_ZYPO.csv *ET* dans /HR/AG04AGR.csv : si le matricule est trouvé dans les 2 fichiers mettre __OK__ sinon __KO__

In [ ]:
# lecture du fichier /BDC/PERIMETRE_CPT_DOSSIER_ZYPO.csv
total_matricules_pos = pd.read_csv('BDC/PERIMETRE_CPT_DOSSIER_ZYPO.csv',sep=';',header=0,usecols=['matcle'])

# Lecture des matricules chargés dans RenoiRH
AG02_RenoiRH = pd.read_csv('HR/MAA_AG04.csv',header=None,names=('matcle','OK'))
# AG02_RenoiRH['OK']='OK'

# création d'une série vérifiant la présence de BdC dans HR
crit_AG04AGR_NC = total_matricules_bdc.matcle.isin(total_matricules_pos.matcle)
crit_AG04AGR_OK = total_matricules_bdc.matcle.isin(AG02_RenoiRH.matcle) & crit_AG01_OK
total_matricules_bdc['PONC']= crit_AG04AGR_NC
total_matricules_bdc['POOK']= crit_AG04AGR_OK
total_matricules_bdc['L4PO']= (total_matricules_bdc['PONC']*1 + total_matricules_bdc['POOK']*1)
# 
total_matricules_bdc['LOT 4 - Positions']=total_matricules_bdc.replace([0, 1, 2], ['NC', 'KO', 'OK'])['L4PO']
# print(total_matricules_bdc)
feuillePerimetre.range('M1').options(index=False).value =total_matricules_bdc['LOT 4 - Positions']


Pour la colonne **PROCHE - Conjoint** rechercher le matricule HR dans /BDC/PERIMETRE_CPT_DOSSIER_PROCHE_CONJOINT.csv *ET* dans /HR/MAA_PR01_CONJ.csv : si le matricule est trouvé dans les 2 fichiers mettre __OK__ sinon __KO__

In [ ]:
# lecture du fichier /BDC/PERIMETRE_CPT_DOSSIER_PROCHE_CONJOINT
total_matricules_pos = pd.read_csv('BDC/PERIMETRE_CPT_DOSSIER_PROCHE_CONJOINT.csv',sep=';',header=0,usecols=['matcle'])

# Lecture des matricules chargés dans RenoiRH
AG02_RenoiRH = pd.read_csv('HR/MAA_PR01_CONJ.csv',header=None,names=('matcle','OK'))
# AG02_RenoiRH['OK']='OK'

# création d'une série vérifiant la présence de BdC dans HR
crit_PR01C_NC = total_matricules_bdc.matcle.isin(total_matricules_pos.matcle)
crit_PR01C_OK = total_matricules_bdc.matcle.isin(AG02_RenoiRH.matcle) & crit_AG01_OK
total_matricules_bdc['PRCNC']= crit_PR01C_NC
total_matricules_bdc['PRCOK']= crit_PR01C_OK
total_matricules_bdc['PR01C']= (total_matricules_bdc['PRCNC']*1 + total_matricules_bdc['PRCOK']*1)
# 
total_matricules_bdc['PROCHE - Conjoint']=total_matricules_bdc.replace([0, 1, 2], ['NC', 'KO', 'OK'])['PR01C']
# print(total_matricules_bdc)
feuillePerimetre.range('N1').options(index=False).value =total_matricules_bdc['PROCHE - Conjoint']


Pour la colonne **PROCHE - Enfants** rechercher le matricule HR dans /BDC/PERIMETRE_CPT_DOSSIER_PROCHE_ENFANT.csv *ET* dans /HR/MAA_PR01_ENF.csv : si le matricule est trouvé dans les 2 fichiers mettre __OK__ sinon __KO__

In [ ]:
# lecture du fic /BDC/PERIMETRE_CPT_DOSSIER_PROCHE_Enfants
total_matricules_pos = pd.read_csv('BDC/PERIMETRE_CPT_DOSSIER_PROCHE_ENFANT.csv',sep=';',header=0,usecols=['matcle'])

# Lecture des matricules chargés dans RenoiRH
AG02_RenoiRH = pd.read_csv('HR/MAA_PR01_ENF.csv',header=None,names=('matcle','OK'))
# AG02_RenoiRH['OK']='OK'

# création d'une série vérifiant la présence de BdC dans HR
crit_PR01E_NC = total_matricules_bdc.matcle.isin(total_matricules_pos.matcle)
crit_PR01E_OK = total_matricules_bdc.matcle.isin(AG02_RenoiRH.matcle) & crit_AG01_OK
total_matricules_bdc['PRENC']= crit_PR01E_NC
total_matricules_bdc['PREOK']= crit_PR01E_OK
total_matricules_bdc['PR01E']= (total_matricules_bdc['PRENC']*1 + total_matricules_bdc['PREOK']*1)
# 
total_matricules_bdc['PROCHE - Enfants']=total_matricules_bdc.replace([0, 1, 2], ['NC', 'KO', 'OK'])['PR01E']
# print(total_matricules_bdc)
feuillePerimetre.range('O1').options(index=False).value =total_matricules_bdc['PROCHE - Enfants']


Pour la colonne **LOT 5 - Absences** rechercher le matricule HR dans /BDC/PERIMETRE_CPT_DOSSIER_ZYAG.csv *ET* dans /HR/AG05AGR.csv : si le matricule est trouvé dans les 2 fichiers mettre __OK__ sinon __KO__

In [ ]:
# lecture du fichier /BDC/PERIMETRE_CPT_DOSSIER_ZYAG.csv
total_matricules_abs = pd.read_csv('BDC/PERIMETRE_CPT_DOSSIER_ZYAG.csv',sep=';',header=0,usecols=['matcle'])

# Lecture des matricules chargés dans RenoiRH
AG02_RenoiRH = pd.read_csv('HR/MAA_AG05.csv',header=None,names=('matcle','OK'))
# AG02_RenoiRH['OK']='OK'

# création d'une série vérifiant la présence de BdC dans HR
crit_AG05AGR_NC = total_matricules_bdc.matcle.isin(total_matricules_abs.matcle)
crit_AG05AGR_OK = total_matricules_bdc.matcle.isin(AG02_RenoiRH.matcle) & crit_AG01_OK
total_matricules_bdc['AGNC']= crit_AG05AGR_NC
total_matricules_bdc['AGOK']= crit_AG05AGR_OK
total_matricules_bdc['L5AG']= (total_matricules_bdc['AGNC']*1 + total_matricules_bdc['AGOK']*1)
# )
total_matricules_bdc['LOT 5 - Absences']=total_matricules_bdc.replace([0, 1, 2], ['NC', 'KO', 'OK'])['L5AG']
# print(total_matricules_bdc)
feuillePerimetre.range('P1').options(index=False).value =total_matricules_bdc['LOT 5 - Absences']
